# TP 02 - Vision por Computadoras I

**Alumno**: Matias Tripode

# 1) Se debe implementar un algoritmo que dada una imagen, o región, calcule la métrica propuesta en el paper "Image Sharpness Measure for Blurred Images in Frequency Domain“ y realizar tres experimentos:

###  1.1) Medición sobre todo el frame.

In [1]:
%matplotlib qt


import numpy as np
import cv2 as cv
import supervision as sv
from skimage.util import random_noise
from matplotlib import pyplot as plt

In [2]:
def _filter_round_shape(height, width, radius):
  canvas = np.zeros((height, width))
  return cv.circle(canvas,
                     center=(int(canvas.shape[1]/2), int(canvas.shape[0]/2)),
                     radius=radius,
                     color=(255),
                     thickness=-1).astype(np.uint8)

In [3]:
def create_hp_filter(height, width, radius=60):

  lp_filter = _filter_round_shape(height, width, radius)
  return cv.bitwise_not(lp_filter)

In [4]:
# Implementacion del Algoritmo explicado en "Image Sharpness Measure for Blurred Images in Frequency Domain"
def calcular_metrica_frequency_domain_blur_measure(frame):
    # Step 1: Compute F which is the Fourier Transform representation of image I
    frame_fft = np.fft.fft2(frame)
    # Step 2: Find Fc which is obtained by shifting the origin of F to centre.
    frame_fft_center = np.fft.fftshift(frame_fft)
    # Step 3: Calculate AF = abs (Fc) where AF is the absolute value of the centered Fourier transform of image I. 
    frame_fft_abs = np.abs(frame_fft_center) 
    # Step 4: Calculate M = max (AF) where M is the maximum value of the frequency component in F. 
    frame_fft_max = np.max(frame_fft_abs)
    # Step 5: Calculate TH = the total number of pixels in F whose pixel value > thres, where thres = M/1000. 
    thres = frame_fft_max / 1000.0
    frame_fft_thres = np.count_nonzero(frame_fft_abs > thres)
    # Step 6: Calculate Image Quality measure (FM) from equation (1).
    return frame_fft_thres/(frame.shape[0] * frame.shape[1])



In [5]:
def calcular_nitidez(frame, algoritmo):
    # Convertir a escala de grises
    frame_gris = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Aplicamos un suavizado Gaussiano de 5x5
    frame_blur = cv.GaussianBlur(frame_gris, (5,5), sigmaX=1.5)
    # Aplicar filtro de realce de bordes con Canny
    bordes = cv.Canny(frame_blur, threshold1=5, threshold2=200)
    # Calcular métrica de nitidez sobre bordes
    nitidez = algoritmo(bordes)
    return nitidez


In [6]:
def obtener_metrica_nitidez_para_todo_el_frame(video_path, algoritmo_metrica_de_enfoque):
    # Abre el video
    captura_video = cv.VideoCapture(video_path)
    if not captura_video.isOpened():
        print("Error al abrir el archivo de video")
    else:
        result = {'count_frame': [], 'nitidez': []}
        count = 0
        while True:
            ret, frame = captura_video.read()
            if ret:
                # Procesar frame
                nitidez = calcular_nitidez(frame, algoritmo=algoritmo_metrica_de_enfoque)
                result['count_frame'].append(count)
                result['nitidez'].append(nitidez)
                count += 1
            else:
                # Si no hay más frames (ret == False), salir del bucle
                break

    # Libera el objeto de captura de video
    captura_video.release()
    cv.destroyAllWindows()
    return result

In [7]:
def graficar_nitidez(datos, titulo, color):
    # Crear la gráfica
    plt.figure(figsize=(10, 5))  # Tamaño del gráfico (opcional)
    plt.plot(datos['count_frame'], datos['nitidez'], 
            marker='o',
            linestyle='',  # Línea continua
            color=color,  # Color de la línea
            label='Nitidez por frame')

    # Personalizar el gráfico
    plt.title(titulo, fontsize=14)
    plt.xlabel('Número de Frame (count_frame)', fontsize=12)
    plt.ylabel('Nitidez', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)  # Cuadrícula
    plt.legend()  # Mostrar leyenda

    # Ajustar límites del eje X para que empiece en 0
    plt.xlim(0, max(datos['count_frame']) + 1)

    # Mostrar la gráfica
    plt.show()

In [8]:
# Funcion para graficar la nitidez. Normalización min-max a [0, 1]
def normalizar_nitidez(nitidez):
    nitidez_min = np.min(nitidez)
    nitidez_max = np.max(nitidez)
    divisor = (nitidez_max - nitidez_min) if (nitidez_max - nitidez_min) != 0 else 1
    nitidez_normalizada = (nitidez - nitidez_min) / divisor
    return nitidez_normalizada

In [9]:
datos = obtener_metrica_nitidez_para_todo_el_frame('focus_video.mov', algoritmo_metrica_de_enfoque=calcular_metrica_frequency_domain_blur_measure)
datos['nitidez'] = normalizar_nitidez(np.array(datos['nitidez']))
# Graficar nitidez vs frame
graficar_nitidez(datos, 'Nitidez para todo el frame, metrica Frequency Domain Blur', color='blue')

-------------

###  1.2) Medición sobre una ROI ubicada en el centro del frame. Area de la ROI = 5 o 10% del area total del frame.

In [10]:
def obtener_region_centrada(frame, porcentaje):
    filas, columnas = frame.shape[:2] 
    # Calcular el tamaño del lado cuadrado de la ROI
    lado_roi = int(min(filas, columnas) * porcentaje)

    # Calcular coordenadas para centrar la ROI
    x_centro, y_centro = columnas // 2, filas // 2
    x1 = x_centro - lado_roi // 2
    y1 = y_centro - lado_roi // 2
    x2 = x1 + lado_roi
    y2 = y1 + lado_roi

    # recortar el cuadrado roi
    roi = frame[y1:y2, x1:x2]
    return roi


In [11]:
def obtener_metrica_nitidez_para_regrion_roi(video_path, porcentaje, algoritmo_metrica_de_enfoque):
    # Abre el video
    captura_video = cv.VideoCapture(video_path)
    if not captura_video.isOpened():
        print("Error al abrir el archivo de video")
    else:
        result = {'count_frame': [], 'nitidez': []}
        count = 0
        while True:
            ret, frame = captura_video.read()            
            if ret:
                region_roi = obtener_region_centrada(frame, porcentaje)
                # Procesar frame
                nitidez = calcular_nitidez(region_roi, algoritmo=algoritmo_metrica_de_enfoque)
                result['count_frame'].append(count)
                result['nitidez'].append(nitidez)
                count += 1
            else:
                # Si no hay más frames (ret == False), salir del bucle
                break

    # Libera el objeto de captura de video
    captura_video.release()
    cv.destroyAllWindows()
    return result

In [12]:
cinco_porciento = 0.05
datos = obtener_metrica_nitidez_para_regrion_roi('focus_video.mov', cinco_porciento, algoritmo_metrica_de_enfoque=calcular_metrica_frequency_domain_blur_measure)
datos['nitidez'] = normalizar_nitidez(np.array(datos['nitidez']))
# Graficar nitidez vs frame
graficar_nitidez(datos, 'Nitidez para region 5%, metrica Frequency Domain Blur', color='blue')

---------------------

# 2). Cambiar la métrica de enfoque eligiendo uno de los algoritmos explicados en el apéndice de: Analysis of focus measure operators in shapefrom focus.

In [13]:
# Calcular metrica de enfoque "Energy of Laplacian (LAP1)" descripta en "Analysis of focus measure operators in shapefrom focus"
def calcular_metrica_energy_of_laplacian(frame):
    # Aplicar el operador Laplaciano
    laplaciano = cv.Laplacian(frame, cv.CV_64F)
    # Calcular la energía del Laplaciano: suma del cuadrado de cada valor
    energia = np.sum(np.square(laplaciano))
    return energia


In [14]:
def calcular_metrica_gray_level_variance(frame):
    # Calcular la varianza
    variance = np.var(frame)
    return variance

In [15]:
datos = obtener_metrica_nitidez_para_todo_el_frame('focus_video.mov', algoritmo_metrica_de_enfoque=calcular_metrica_energy_of_laplacian)
datos['nitidez'] = normalizar_nitidez(np.array(datos['nitidez']))
# Graficar nitidez vs frame
graficar_nitidez(datos, 'Nitidez para todo el frame, metrica Energy of Laplacian (LAP1)', color='green')

In [16]:
cinco_porciento = 0.05
datos = obtener_metrica_nitidez_para_regrion_roi('focus_video.mov', cinco_porciento, algoritmo_metrica_de_enfoque=calcular_metrica_energy_of_laplacian)
datos['nitidez'] = normalizar_nitidez(np.array(datos['nitidez']))
# Graficar nitidez vs frame
graficar_nitidez(datos, 'Nitidez para region 5%, metrica Energy of Laplacian (LAP1)', color='green')